In [1]:
import pandas as pd
from pymongo import MongoClient
from bs4 import BeautifulSoup
import json
from pprint import pprint
import numpy as np

In [2]:
df = pd.read_json('../data/yoga_uncleaned.json')

In [3]:
df

,_id,id,snippet,contentDetails,statistics
0,{'$oid': '5fe47eb4ac534cfd98f0cd13'},sTANio_2E0Q,{'title': '20 min Full Body STRETCH/YOGA for S...,{'duration': 'PT20M56S'},"{'viewCount': '5533960', 'likeCount': '105260'..."
1,{'$oid': '5fe47eb4ac534cfd98f0cd14'},GLy2rYHwUqY,{'title': 'Total Body Yoga - Deep Stretch | Yo...,{'duration': 'PT45M13S'},"{'viewCount': '13443080', 'likeCount': '131111..."
2,{'$oid': '5fe47eb4ac534cfd98f0cd15'},4pKly2JojMw,{'title': '10 min Morning Yoga Full Body Stret...,{'duration': 'PT10M56S'},"{'viewCount': '18857966', 'likeCount': '339861..."
3,{'$oid': '5fe47eb5ac534cfd98f0cd16'},b1H3xO3x_Js,{'title': 'Full Body Flow | 20 Min. Yoga Pra...,{'duration': 'PT19M18S'},"{'viewCount': '6541616', 'likeCount': '65820',..."
4,{'$oid': '5fe47eb5ac534cfd98f0cd17'},A0pkEgZiRG4,"{'title': '10 Minute Yoga Full Body Stretch', ...",{'duration': 'PT10M7S'},"{'viewCount': '705284', 'likeCount': '13947', ..."
...,...,...,...,...,...
14614,{'$oid': '5fe490b85b79cfe119f631c3'},1YVtvyUNNTI,{'title': '30DYC: 30 Day Yoga Challenge by Das...,{'duration': 'PT52M10S'},"{'likeCount': '0', 'dislikeCount': '0', 'comme..."
14615,{'$oid': '5fe490b85b79cfe119f631c4'},2vodbbORTvY,{'title': '30DYC: 30 Day Yoga Challenge by Das...,{'duration': 'PT55M42S'},"{'likeCount': '1', 'dislikeCount': '0', 'comme..."
14616,{'$oid': '5fe490b85b79cfe119f631c5'},iuIP4vZ8bUw,{'title': '30DYC: 30 Day Yoga Challenge by Das...,{'duration': 'PT34M'},"{'likeCount': '0', 'dislikeCount': '0', 'comme..."
14617,{'$oid': '5fe490b95b79cfe119f631c6'},ogLP-P7qz4g,"{'title': 'Acusada', 'description': 'A young s...",{'duration': 'PT1H53M14S'},"{'likeCount': '0', 'dislikeCount': '0', 'comme..."


In [21]:
def split_columns(data):
    df = pd.read_json(data)

    # access dictionary values, create new columns and drop parents
    snippet = []
    contentDetails = []
    statistics = []
    columns = ['snippet', 'contentDetails', 'statistics']
    new_dfs = [snippet, contentDetails, statistics]
    for item, new_df in zip(columns, new_dfs):
        new_df.append(pd.json_normalize(df[item]))
    new_df = pd.concat([df, snippet[0], contentDetails[0], statistics[0]], axis=1)
    new_df.drop(columns=columns, inplace=True)
    new_df.drop(columns=['_id'], inplace=True)
    new_df.set_index('id', inplace=True)
    
    # un-nest 'tags' column
    new_df['tags'] = new_df['tags'].apply(lambda x: ', '.join(map(str, x)) if type(x)==list else np.nan)
    
    # cast numeric string values to float, pad NaN values
    columns_ = ['categoryId', 'viewCount', 'likeCount', 'dislikeCount', 'commentCount']
    for column in columns_:
        new_df[column] = new_df[column].apply(lambda x: int(x) if type(x)==str else np.nan)
        new_df[column].fillna(method='pad', inplace=True)
    
    return new_df

In [22]:
data = '../data/yoga_uncleaned.json'
test = split_columns(data)

In [194]:
test.loc[~test['tags'].isnull()][['tags']]

,tags
id,
sTANio_2E0Q,"20 min full body stretch, full body stretch, s..."
GLy2rYHwUqY,"total body yoga, yoga practice, yoga at home, ..."
4pKly2JojMw,"10 minute morning yoga, morning yoga routine, ..."
b1H3xO3x_Js,"free yoga videos, yoga at home, morning yoga, ..."
A0pkEgZiRG4,"yoga, yoga full body stretch, full body stretc..."
...,...
tCIKpsuvWwU,iMovie
k82S4ND24Fk,"yoga flow workout, vinyasa flow yoga, yoga flo..."
tFkHgtPN8lA,"#yogaflow, #traumainformedyoga, #vinyasayoga, ..."


In [192]:
test['tags'] = test['tags'].apply(lambda x: ', '.join(map(str, x)) if type(x)==list else np.nan)

In [220]:
test['viewCount'] = test['viewCount'].apply(lambda x: int(x) if type(x)==str else np.nan)

In [23]:
test.describe()

,categoryId,viewCount,likeCount,dislikeCount,commentCount
count,14619.000000,1.461900e+04,14619.000000,14619.000000,14619.000000
mean,21.992134,3.562409e+05,3377.504891,151.403311,147.797113
std,5.766908,2.939024e+06,19723.354418,3032.415039,670.327247
min,1.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,22.000000,4.605000e+02,14.000000,0.000000,1.000000
50%,22.000000,9.594000e+03,183.000000,4.000000,15.000000
75%,26.000000,7.171800e+04,1224.000000,28.000000,80.000000
max,30.000000,1.571291e+08,682484.000000,270853.000000,26329.000000


In [8]:
copy = test.copy()

In [24]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14619 entries, sTANio_2E0Q to sFlK4kh3nUQ
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         14619 non-null  object 
 1   description   14619 non-null  object 
 2   channelTitle  14619 non-null  object 
 3   tags          11971 non-null  object 
 4   categoryId    14619 non-null  int64  
 5   duration      14619 non-null  object 
 6   viewCount     14619 non-null  float64
 7   likeCount     14619 non-null  float64
 8   dislikeCount  14619 non-null  float64
 9   commentCount  14619 non-null  float64
dtypes: float64(4), int64(1), object(5)
memory usage: 1.2+ MB


In [262]:
copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14619 entries, sTANio_2E0Q to sFlK4kh3nUQ
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         14619 non-null  object 
 1   description   14619 non-null  object 
 2   channelTitle  14619 non-null  object 
 3   tags          11971 non-null  object 
 4   categoryId    14619 non-null  int64  
 5   duration      14619 non-null  object 
 6   viewCount     14503 non-null  float64
 7   likeCount     14406 non-null  float64
 8   dislikeCount  14406 non-null  float64
 9   commentCount  14275 non-null  float64
dtypes: float64(4), int64(1), object(5)
memory usage: 1.5+ MB


In [17]:
test.loc['viewCount', 'likeCount', 'dislikeCount', 'commentCount'].fillna(method='pad', inplace=True)

IndexingError: Too many indexers

In [20]:
test['viewCount'].fillna(method='pad', inplace=True)

In [12]:
copy.median()

categoryId        22.0
viewCount       9531.0
likeCount        186.0
dislikeCount       4.0
commentCount      16.0
dtype: float64

In [271]:
copy[['viewCount']].map(lambda x: type(x))

AttributeError: 'DataFrame' object has no attribute 'map'

In [ ]:
copy.fillna()

In [260]:
test.describe()

,categoryId,viewCount,likeCount,dislikeCount,commentCount
count,14619.000000,1.450300e+04,14406.000000,14406.000000,14275.000000
mean,21.992134,3.383748e+05,3409.710398,152.891989,147.899615
std,5.766908,2.787684e+06,19861.525343,3054.535920,670.035156
min,1.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,22.000000,4.620000e+02,14.000000,0.000000,1.000000
50%,22.000000,9.531000e+03,186.000000,4.000000,16.000000
75%,26.000000,7.050100e+04,1238.750000,28.000000,80.000000
max,30.000000,1.571291e+08,682484.000000,270853.000000,26329.000000
